<a href="https://colab.research.google.com/github/Bjegsad/Kontur/blob/main/Kontur_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets evaluate transformers
!pip install accelerate
!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [2]:
!git config --global user.email 'ya.chepurin1809@gmail.com'
!git config --global user.name 'YuriChep'

# Preprocessing

In [3]:
#Все библиотеки
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import collections
from tqdm.auto import tqdm
import numpy as np
import evaluate
from huggingface_hub import notebook_login
from transformers import TrainingArguments
from transformers import Trainer

In [53]:
df = load_dataset('json', data_files = 'content/train.json', split = 'train') #загружаем наш датасет
df = df.train_test_split(test_size = 0.3) #делим на трейн и тест выборку
df #посмотрим, что получилось

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'extracted_part'],
        num_rows: 1205
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'extracted_part'],
        num_rows: 594
    })
})

In [35]:
train = df['train'] #тренировочная выборка
test = df['test'] #тестовая выборка (в трейновом датасете)


In [36]:
questions = ["Какой фрагмент текста соответствует размеру {}?".format(label) for label in train['label']]
train = train.remove_columns("label").add_column("question", questions)
train[1]
val_questions = ["Какой фрагмент текста соответствует размеру {}?".format(label) for label in test['label']]
test = test.remove_columns("label").add_column("question", val_questions)

Flattening the indices:   0%|          | 0/1205 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/594 [00:00<?, ? examples/s]

In [37]:
#Посмотрим на данные
print("Context: ", train[2]["text"])
print("Question: ", train[2]["question"])
print("Answer: ", train[2]['extracted_part'])

Context:  Извещение о проведении запроса котировок в электронной форме для закупки №0320200019822000048 Общая информация Номер извещения 0320200019822000048 Наименование объекта закупки Поставка стеллажа складского металлического Способ определения поставщика (подрядчика, исполнителя) Запрос котировок  или возмещает расходы на их исправление потребителем или третьим лицом. Устранение недостатков товара или возмещение расходов на их исправление производится за счет Поставщика, если эти недостатки возникли не по вине Заказчика. Расходы на устранение недостатков товаров в гарантийный срок – несет Поставщик за счет собственных средств. Требования к гарантии производителя товара гарантия поставщика на товар должна быть не менее срока гарантии производителя продукции Срок, на который предоставляется гарантия Гарантийный срок эксплуатации — не менее 12 месяцев с момента ввода стеллажей в эксплуатацию (даты подписания Заказчиком документа о приемке). Указанный срок может быть продлен в случае 

In [38]:
tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence') #https://habr.com/ru/articles/562064/

In [39]:
max_length = 384
stride = 128
def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["text"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["extracted_part"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_end"][0]
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["text"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [40]:
train_dataset = train.map(
    preprocess_training_examples,
    batched = True,
    remove_columns=train.column_names,
)
len(train), len(train_dataset)

Map:   0%|          | 0/1205 [00:00<?, ? examples/s]

(1205, 2374)

In [41]:
test_dataset = test.map(
    preprocess_validation_examples,
    batched = True,
    remove_columns = train.column_names,
)
len(test), len(test_dataset)

Map:   0%|          | 0/594 [00:00<?, ? examples/s]

(594, 1150)

# Compute Metrics

In [42]:
n_best = 20
max_answer_length = 150
metric = evaluate.load('squad')

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["text"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": str(example_id), "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": str(ex["id"]), "answers": {'text': ex["extracted_part"]['text'], 'answer_start': ex["extracted_part"]['answer_start']}} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

# Training

In [43]:
model = AutoModelForQuestionAnswering.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:


notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [45]:


args = TrainingArguments(
    "DeepPavlov/rubert-base-cased-sentence",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=4,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=True,
)

In [46]:


trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)
trainer.train()

Cloning https://huggingface.co/YuryCHep/rubert-base-cased-sentence into local empty directory.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.983400
1000,0.262700


TrainOutput(global_step=1188, training_loss=0.5525839577620278, metrics={'train_runtime': 499.2922, 'train_samples_per_second': 19.019, 'train_steps_per_second': 2.379, 'total_flos': 1860955501473792.0, 'train_loss': 0.5525839577620278, 'epoch': 4.0})

In [47]:
tokenizer = AutoTokenizer.from_pretrained('YuryCHep/rubert-base-cased-sentence')
model = AutoModelForQuestionAnswering.from_pretrained('YuryCHep/rubert-base-cased-sentence')

In [48]:
predictions, _, _ = trainer.predict(test_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, test_dataset, test)

  0%|          | 0/594 [00:00<?, ?it/s]

{'exact_match': 65.65656565656566, 'f1': 77.99262618398251}

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained('/content/sberbank-ai/sbert_large_nlu_ru/checkpoint-1164')